In [1]:
from data.geographic.arcgis import getStockData
from data.geographic.shapefileGeo import testSQLAlchemyORM, shapefileTest, generateWorldToDB
from data.geographic.geo import AddInformationUsingDataFrame, GisMapUpdate
from data.geographic.geo2 import makeMapLayer

import pandas as pd
import geopandas as gp
from shapely.geometry import Point

In [2]:
df_main = getStockData()


    shapefile
    ---------------
    - test to print shapefile content 
    - divided to two files dbf and shp
    - uses dictionaries as resultsets to contain data related to location and the location as polycon
    
    Using datasets geopandas for country and city statistics OR Using the gadm28 dataset  
    - http://stackoverflow.com/questions/31997859/bulk-insert-a-pandas-dataframe-using-sqlalchemy
    
    crs (coordinate system )
    
    http://stackoverflow.com/questions/3845006/database-of-countries-and-their-cities


In [2]:
'''
Load location data using pre-existing routines
'''
naturalEarthToCSV = True
    
if naturalEarthToCSV:
    # to load into DB reverse getAsPandasDataFrame flag 
    gp_world, gp_cities = generateWorldToDB(loadCSV = True, getAsPandasDataFrame = False)  
    
    print ('Countries: ', gp_world)
    print ('Cities: ', gp_cities)

Countries:                     continent  gdp_md_est  \
0                       Asia     22270.0   
1                     Africa    110300.0   
2                     Europe     21810.0   
3                       Asia    184300.0   
4              South America    573900.0   
5                       Asia     18770.0   
6                 Antarctica       760.4   
7    Seven seas (open ocean)        16.0   
8                    Oceania    800200.0   
9                     Europe    329500.0   
10                      Asia     77610.0   
11                    Africa      3102.0   
12                    Europe    389300.0   
13                    Africa     12830.0   
14                    Africa     17820.0   
15                      Asia    224000.0   
16                    Europe     93750.0   
17             North America      9093.0   
18                    Europe     29700.0   
19                    Europe    114100.0   
20             North America      2536.0   
21             South

In [9]:

'''
all available fields:  ['OBJECTID', 'UID', 'ID_0', 'ISO', 'NAME_0', 
'ID_1', 'NAME_1', 'VARNAME_1', 'NL_NAME_1', 'HASC_1', 'CCN_1', 'CCA_1', 'TYPE_1', 'ENGTYPE_1', 'VALIDFR_1', 'VALIDTO_1', 'REMARKS_1', 
'ID_2', 'NAME_2', 'VARNAME_2', 'NL_NAME_2', 'HASC_2', 'CCN_2', 'CCA_2', 'TYPE_2', 'ENGTYPE_2', 'VALIDFR_2', 'VALIDTO_2', 'REMARKS_2', 
'ID_3', 'NAME_3', 'VARNAME_3', 'NL_NAME_3', 'HASC_3', 'CCN_3', 'CCA_3', 'TYPE_3', 'ENGTYPE_3', 'VALIDFR_3', 'VALIDTO_3', 'REMARKS_3', 
'ID_4', 'NAME_4', 'VARNAME_4', 'CCN_4', 'CCA_4', 'TYPE_4', 'ENGTYPE_4', 'VALIDFR_4', 'VALIDTO_4', 'REMARKS_4', 
'ID_5', 'NAME_5', 'CCN_5', 'CCA_5', 'TYPE_5', 'ENGTYPE_5', 'REGION', 'VARREGION', 'Shape_Leng', 'Shape_Area']
'''

loadShapefileData = False         # load shapefile content as dictionary from i instance to i_max - slow
esriShapefileToGeopandas = True   # use geopandas to read shapefile to Dataframe - fast

if loadShapefileData:
    shapefileTest(i = 0, i_max = 1)

if esriShapefileToGeopandas:
    '''
    Usefull fields 
    'OBJECTID', 'geometry', 'UID', 'ID_0', 'ISO', 'NAME_0', 
         'REGION', 'VARREGION', 'Shape_Leng', 'Shape_Area'
         
         'ID_1', 'NAME_1', 
         'ID_2', 'NAME_2', 
         'ID_3', 'NAME_3', 
         'ID_4', 'NAME_4', 
         'ID_5', 'NAME_5', 
    '''
    shp = gp.GeoDataFrame.from_file('./gadm28/gadm28.shp')
    shp_1 = shp[['OBJECTID', 'geometry']]
    shp = shp[['OBJECTID', 'UID', 'ID_0', 'ISO', 'NAME_0', 'REGION', 
                   'VARREGION', 'Shape_Leng', 'Shape_Area', 'ID_1', 'NAME_1','ID_2', 'NAME_2',
                   'ID_3', 'NAME_3', 'ID_4', 'NAME_4', 'ID_5', 'NAME_5']]
 
    #save X,Y into csv file
    #shp.to_csv("./data/allData.csv",header=True,index=False,sep="\t")
    #shp_1.to_csv("./data/allData_geom.csv",header=True,index=False,sep="\t")

In [4]:
'''
Combine, transpose and store data stored into dataframe 

    cities: Country,City,AccentCity,Region,Population,Latitude,Longitude
        - Country, City, Population,Latitude,Longitude - link to add iso3 
        
    countrycodes: euname,modified,linked_country,iso3,iso2,grc,isonum,country,imperitive
        - country, iso3, iso2
            
    - define datasets
    - merge with country
    - add geometry  
    - store to csv  
'''  


combineDataForCities = True        
if combineDataForCities: 
    df_cities = pd.read_csv("./data/worldcitiespop.csv", sep = ',', encoding = "ISO-8859-1", header = 0,  
                            names=['Country','City','AccentCity','Region','Population','Latitude','Longitude'])
    df_cities = df_cities[['Country','City','Region','Population','Latitude','Longitude']]
    df_cities.columns = ['iso2', 'City','Region','Population','Latitude','Longitude']
    df_cities['iso2'] = df_cities['iso2'].str.upper()
    df_cities = df_cities[df_cities['Population'] > 50000]
        
    df_countryCodes = pd.read_csv("./data/countryISO2, 3.csv", sep = ',', header = 0,
                                  names=['euname','modified','linked_country','iso3','iso2','grc','isonum','country','imperitive'])
    df_countryCodes = df_countryCodes[['country', 'iso3', 'iso2']]
        
    df_main = pd.merge(df_cities, df_countryCodes, on='iso2', how='inner')
        

    geometry = [Point(xy) for xy in zip(df_main.Longitude, df_main.Latitude)]
    crs = {'init': 'epsg:4326'}
    df_geo = gp.GeoDataFrame(df_main, crs=crs, geometry=geometry)


C:\python\New folder\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
'''
df_main contains the end-result used as the arcgis feature-layer. 
It contains the share prise indexed with the company symbol with daily adjusted close column used as changeable values 
    in fixex table. The constraint for fixed table setup is required by arcgis.
The dataframe merges location data and stock data from separate sources.
'''
df_main

,0,a,b,c,d,e,f,g,h,Latitude,Longitude
symbol,,,,,,,,,,,
A,45.298838,46.076585,45.907077,45.837280,46.066615,46.375718,45.588002,45.639999,45.560001,37.354168,-121.954170
AAL,48.580002,49.220001,49.470001,48.689999,48.480000,48.610001,47.669998,47.250000,46.689999,32.725277,-97.320557
AAP,173.760621,175.570000,173.509995,170.389999,170.889999,171.839996,170.419998,170.279999,169.119995,37.270832,-79.941666
AAPL,116.639999,116.949997,117.059998,116.290001,116.519997,117.260002,116.760002,116.730003,115.820000,37.323055,-122.031113
ABBV,61.598480,61.539078,60.836172,61.044073,61.717281,61.875682,61.647980,62.103384,61.994483,41.849998,-87.650002
ABT,38.093934,37.944896,37.994578,38.044256,38.173419,38.352264,37.974706,38.064128,38.163485,41.849998,-87.650002
ACN,123.650002,124.099998,117.900002,117.790001,117.480003,117.550003,116.610001,117.010002,117.129997,53.333057,-6.248889
ADBE,105.290001,105.769997,105.510002,104.720001,105.019997,104.980003,103.769997,103.680000,102.949997,37.339443,-121.893890
ADM,46.070000,46.220001,46.130001,45.180000,45.150002,45.650002,45.340000,45.360001,45.650002,34.605835,-86.983330
